In [1]:
import alpaca_trade_api as tradeapi
import time
import threading
import csv
import pandas as pd

# Alpaca API credentials
API_KEY = 'PKL0QPAXI2XEQN42OD2S'
API_SECRET = 'nkqziIUvRn7rkEihNRVpw9IcNYftpDxVHPRQnyS3'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# Main csv file
csv_file = "financial_news.csv"

# News data storage
all_news = []

# List of tickers
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB", "INTC", "TSLA", "NVDA", "ORCL", "IBM",
    "JPM", "BAC", "WFC", "C", "GS", "AXP", "MS", "BRK.B",
    "JNJ", "PFE", "UNH", "MRK", "ABBV", "TMO", "ABT",
    "PG", "KO", "PEP", "NKE", "MO",
    "GE", "MMM", "BA", "HON", "CAT",
    "XOM", "CVX", "BP", "TOT",
    "NEP", "DUK", "SO",
    "T", "VZ",
    "DIS", "MCD", "SBUX",
    "WMT", "HD"
]

Data Collection

In [2]:
# Function to fetch news for a ticker
def fetch_news(ticker):
    global all_news
    try:
        news = api.get_news(symbol=ticker, limit=200)  # Adjust the limit as needed
        
        for article in news:
            all_news.append({
                'headline': article.headline,
                'summary': article.summary,
                'created_at': article.created_at,
                'ticker': ticker
            })
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")

# Function to process requests in batches
def process_requests():
    while tickers:
        threads = []
        for _ in range(200):  # 200 parallel threads, respecting the rate limit
            if tickers:
                ticker = tickers.pop(0)
                thread = threading.Thread(target=fetch_news, args=(ticker,))
                threads.append(thread)
                thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        # Wait for 60 seconds before the next batch
        time.sleep(60)

# Write data to CSV
def write_to_csv():
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=all_news[0].keys())
        writer.writeheader()
        for news_item in all_news:
            writer.writerow(news_item)

    print(f'Data saved to {csv_file}')

Data Cleaning Functions

In [3]:
import re

# Keep digits and symbols that can be significant in financial analysis. 
# E.g. digits could represent financial figures, and symbols like '$' might indicate currency amounts. 
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)  
    # Remove newlines and tabs
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ') 
#     # Keep words, digits, $ and .
#     text = re.sub(r'[^\w\d$.]', ' ', text)  
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)  
    return text.strip()

In [4]:
'''
Testing clean_data()
'''
# Example DataFrame
data = pd.DataFrame({
    'article': ["Stocks rallied, with AAPL reaching $150.50. More at http://example.com", 
                "Q1 profits fell to $1.5 billion, down from $2 billion. Visit http://finance.com for more.", 
                "Tech stocks, e.g., MSFT, AMZN, and GOOG, showed mixed results today."]
})

data['cleaned_article'] = data['article'].apply(clean_text)
print(data)

                                             article  \
0  Stocks rallied, with AAPL reaching $150.50. Mo...   
1  Q1 profits fell to $1.5 billion, down from $2 ...   
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...   

                                     cleaned_article  
0  Stocks rallied, with AAPL reaching $150.50. Mo...  
1  Q1 profits fell to $1.5 billion, down from $2 ...  
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...  


In [5]:
from datetime import datetime
def extract_date(date_str):
    try:
        # Parse the string to a datetime object
        dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S+00:00')
        # Format the datetime object to keep only the date part
        return dt.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None

In [6]:
'''
Testing extract_date()
'''
date_string = "2024-01-25 13:37:00+00:00"
extracted_date = extract_date(date_string)
print(extracted_date)

2024-01-25


In [7]:
# # Gather data and write to csv file
# process_requests()
# write_to_csv()

Data Cleaning 

In [8]:
df = pd.read_csv(csv_file)
df = df.dropna().reset_index(drop=True)

df['headline'] = df['headline'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)
df['created_at'] = df['created_at'].apply(extract_date)
df['headline_summary'] = df['headline'] + " " + df['summary']

df

,headline,summary,created_at,ticker,headline_summary
0,How We Missed A Winning Trade On IBM--And How ...,We Won&#39;t Make This Mistake Again,2024-01-26,IBM,How We Missed A Winning Trade On IBM--And How ...
1,Investigating IBM's Standing In IT Services In...,,2024-01-26,IBM,Investigating IBM's Standing In IT Services In...
2,90's Tech Titans Are Outshining 'Magnificent S...,Companies that have strong cash flows will lik...,2024-01-26,IBM,90's Tech Titans Are Outshining 'Magnificent S...
3,Why Humana Shares Are Trading Lower By Over 11...,Shares of Humana Inc. (NYSE: HUM) fell sharply...,2024-01-25,IBM,Why Humana Shares Are Trading Lower By Over 11...
4,"S&P 500 Eyes Sixth Green Session, Tesla Tumble...",U.S. stock market gains due to strong economic...,2024-01-25,IBM,"S&P 500 Eyes Sixth Green Session, Tesla Tumble..."
...,...,...,...,...,...
6798,"Earnings Scheduled For July 30, 2020",Companies Reporting Before The Bell • Masterca...,2020-07-30,TOT,"Earnings Scheduled For July 30, 2020 Companies..."
6799,13 Energy Stocks Moving In Tuesday's Pre-Marke...,"Gainers • Laredo Petroleum, Inc. (NYSE:LPI) st...",2020-06-02,TOT,13 Energy Stocks Moving In Tuesday's Pre-Marke...
6800,"Earnings Scheduled For May 5, 2020",Companies Reporting Before The Bell L3Harris T...,2020-05-05,TOT,"Earnings Scheduled For May 5, 2020 Companies R..."
6801,"Earnings Scheduled For April 30, 2020",Companies Reporting Before The Bell McDonald&#...,2020-04-30,TOT,"Earnings Scheduled For April 30, 2020 Companie..."


In [9]:
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')

# Tokenisation
df['tokens'] = df['headline_summary'].apply(word_tokenize)

In [10]:
# Removing Stop Words
from nltk.corpus import stopwords
# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

In [11]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['filtered_tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

In [12]:
# Save to csv
df.to_csv('financial_news_cleaned.csv', index=False)